In [11]:
import os
os.chdir('..')

In [12]:
import logging
from pathlib import Path
import h5py

import numpy as np
import tensorflow as tf

In [13]:
data_dir = Path("data/emnist_dig_fed")
is_extract = not data_dir.exists()
# if not is_extract:
#     return

logging.info(f"Data extration: {is_extract} ...")
filename = 'fed_emnist_digitsonly.tar.bz2'
path = tf.keras.utils.get_file(
  filename,
  origin='https://storage.googleapis.com/tff-datasets-public/' + filename,
  file_hash='55333deb8546765427c385710ca5e7301e16f4ed8b60c1dc5ae224b42bd5b14b',
  hash_algorithm='sha256',
  extract=True,
  archive_format='tar')
logging.info(f"Data extrated")

dir_path = os.path.dirname(path)

def download_split(data_set):
    split_dir = data_dir / data_set
    try:
        os.makedirs(split_dir)
    except FileExistsError:
        return
    h5 = h5py.File(os.path.join(dir_path, f"fed_emnist_digitsonly_{data_set}.h5"), "r")[
        "examples"
    ]

    for client_id in h5.keys():
        images = np.expand_dims(h5[client_id]['pixels'], axis=1)
        labels = np.array(h5[client_id]['label'])
        np.save(split_dir / f"{client_id}_img", images)
        np.save(split_dir / f"{client_id}_label", labels)

download_split("train")
download_split("test")

In [14]:
def get_client_data(data_set, client_id):
    data_dir = Path("data/emnist_dig_fed")
    split_dir = data_dir / data_set

    images = np.load(split_dir / f"{client_id}_img.npy")
    labels = np.load(split_dir / f"{client_id}_label.npy")
    return images, labels


def get_data(split):
    data_dir = Path("data/emnist_dig_fed")
    if isinstance(split, str) and split == "test":
        data_set = "test"
        ext = '_img.npy'
        client_ids = [f[:-len(ext)] for f in os.listdir(data_dir/data_set) if f.endswith(ext)]
    else:
        data_set = "train"
        client_ids = split

    images = []
    labels = []
    for c in client_ids:
        i, l = get_client_data(data_set, c)
        images.append(i)
        labels.append(l)

    return np.concatenate(images), np.concatenate(labels)

In [15]:
img, l = get_data(["f0000_14"])

In [16]:
img.shape

(93, 1, 28, 28)

In [ ]:
len(h5.keys())

In [ ]:
from torchvision import datasets
train_ds = datasets.EMNIST("data/emnist", "byclass", download=True,)

In [ ]:
train_ds

In [ ]:
len(np.unique(train_ds.train_labels))

In [3]:
from FLF.data.TorchEMNISTFed import TorchEMNISTFed

In [4]:
ds = TorchEMNISTFed("test")

In [5]:
ids = TorchEMNISTFed.get_client_ids('train')

In [ ]:
len(ids)

In [10]:
len(TorchEMNISTFed(ids))

671585